In [ ]:
import tensorflow as tf
#from tensorflow.keras import Model
import numpy as np
import glob
tf.__version__

'1.15.0'

In [ ]:
IMAGE_SIZE = 224
def representative_data_gen():
    dataset_list=glob.glob('./sample/*/*/*')
    dataset_list_index=np.random.choice(range(len(dataset_list)),100)
    for i in range(100):
        image = dataset_list[dataset_list_index[i]]
        print(image)
        image = tf.io.read_file(image)
        image = tf.io.decode_jpeg(image,channels=3)
        image = tf.image.resize(image,[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image,tf.float32)
        image = tf.expand_dims(image,0)
        with tf.Session() as sess:
            image = sess.run(image)
        yield [image]

In [ ]:
save_keras_model = tf.keras.models.load_model('./model-resnet18.h5',custom_objects={'tf':tf})
save_keras_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 56, 56, 64)   36928       max_pooling2d[0][0]              
______________________________________________________________________________________________

In [ ]:
if tf.__version__[0]=='2':
    converter = tf.lite.TFLiteConverter.from_keras_model(save_keras_model)
elif tf.__version__[0]=='1':
    converter = tf.lite.TFLiteConverter.from_keras_model_file('./model-resnet18.h5')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8 
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen 
tflite_model = converter.convert()

INFO:tensorflow:Froze 106 variables.
INFO:tensorflow:Converted 106 variables to const ops.
./sample\train\car\n02814533_3117.JPEG
./sample\valid\ox\n02403003_10641.JPEG
./sample\train\ox\n02403003_3260.JPEG
./sample\train\dogs\n02109961_3490.JPEG
./sample\train\dogs\n02109961_2931.JPEG
./sample\train\bike\n03792782_5670.JPEG
./sample\train\bike\n03792782_6872.JPEG
./sample\train\bike\n03792782_242.JPEG
./sample\train\ox\n02403003_1196.JPEG
./sample\valid\bike\n03792782_22057.JPEG
./sample\train\cats\n02123159_587.JPEG
./sample\train\dogs\n02109961_3372.JPEG
./sample\valid\dogs\n02109961_9226.JPEG
./sample\train\car\n02814533_13163.JPEG
./sample\train\dogs\n02109961_2518.JPEG
./sample\valid\car\n02814533_73958.JPEG
./sample\train\dogs\n02109961_444.JPEG
./sample\train\car\n02814533_6660.JPEG
./sample\train\ox\n02403003_1997.JPEG
./sample\valid\ox\n02403003_10219.JPEG
./sample\train\cats\n02123159_3002.JPEG
./sample\train\car\n02814533_12268.JPEG
./sample\train\car\n02814533_8757.JPEG
./

In [ ]:
with open('resnet18.tflite','wb') as f:
    f.write(tflite_model)